In [1]:
#PLOTLY NETWORK GRAPH

import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import plotly.graph_objects as go


# Read CSV file
data = pd.read_csv("path/odsi_partner_data_raw.csv") #Download from github and replace with your filepath

# Create an empty graph
G = nx.Graph()

# Iterate through the data and add nodes and edges to the graph
# First, create all the ODSI nodes
for index, row in data.iterrows():
    odsi = row["ODSI"]
    G.add_node(odsi, node_type="ODSI")

# Now, create the partner nodes and edges
for index, row in data.iterrows():
    odsi = row["ODSI"]
    has_partners = row["Partners"]
    list_of_partners = row["List of Partners"]

    if has_partners.lower() == "yes":
        partners = list_of_partners.split(";")
        for partner in partners:
            partner = partner.strip()

            # Only create a partner node if it doesn't already exist as an ODSI node
            if partner not in G.nodes or G.nodes[partner]["node_type"] != "ODSI":
                G.add_node(partner, node_type="partner")

            # Add edge between ODSI and partner
            G.add_edge(odsi, partner)


# Get positions of the nodes using a layout algorithm
pos = nx.spring_layout(G)

# Extract node positions (x and y coordinates) for ODSIs and partners
odsi_x = [pos[n][0] for n, attr in G.nodes(data=True) if attr["node_type"] == "ODSI"]
odsi_y = [pos[n][1] for n, attr in G.nodes(data=True) if attr["node_type"] == "ODSI"]
partner_x = [pos[n][0] for n, attr in G.nodes(data=True) if attr["node_type"] == "partner"]
partner_y = [pos[n][1] for n, attr in G.nodes(data=True) if attr["node_type"] == "partner"]

# Create edge traces for the graph
edge_traces = []
for edge in G.edges():
    x0, y0 = pos[edge[0]]
    x1, y1 = pos[edge[1]]
    edge_traces.append(go.Scatter(x=[x0, x1, None], y=[y0, y1, None], mode='lines', line=dict(color='gray', width=1)))

# Create node traces for ODSIs and partners
odsi_trace = go.Scatter(
    x=odsi_x,
    y=odsi_y,
    mode='markers',
    hovertemplate='%{text}',
    text=[n for n, attr in G.nodes(data=True) if attr["node_type"] == "ODSI"], 
    marker=dict(
        size=15,
        color='blue', 
        line=dict(color='black', width=0.5)
    )
)
partner_trace = go.Scatter(
    x=partner_x, 
    y=partner_y, 
    mode='markers', 
    hovertemplate='%{text}', 
    text=[n for n, attr in G.nodes(data=True) if attr["node_type"] == "partner"], 
    marker=dict(
        size=10, 
        color='red', 
        line=dict(color='black', width=0.5)
    )
)
odsi_label_trace = go.Scatter(x=odsi_x, y=odsi_y, mode='text', text=[n for n, attr in G.nodes(data=True) if attr["node_type"] == "ODSI"], textposition="bottom center", hoverinfo='skip', showlegend=False, marker=dict(size=0, color='rgba(0,0,0,0)'), textfont=dict(size=8))
partner_label_trace = go.Scatter(x=partner_x, y=partner_y, mode='text', text=[n for n, attr in G.nodes(data=True) if attr["node_type"] == "partner"], textposition="bottom center", hoverinfo='skip', showlegend=False, marker=dict(size=0, color='rgba(0,0,0,0)'), textfont=dict(size=8))

# Create the interactive figure
fig = go.Figure(edge_traces + [odsi_trace, partner_trace, odsi_label_trace, partner_label_trace])


# Set the axis properties and layout
fig.update_xaxes(showticklabels=False, showgrid=False, zeroline=False)
fig.update_yaxes(showticklabels=False, showgrid=False, zeroline=False)
fig.update_layout(showlegend=False, hovermode='closest', margin=dict(l=0, r=0, b=0, t=0))



# Show the interactive network graph
fig.show()
